In [2]:
import bs4
import lxml
import pandas as pd
import urllib
import re

from urllib import request

base_url = "https://howlongtobeat.com/game/"

c:\Users\rapha\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
def get_title(page):
    title = page.find("title").text
    title = title[12:].split('?')[0]
    return title

def get_rating(page):
    rating_tag = page.find("a", attrs={"class": "text_primary", "href": re.compile("/game/[0-9]+/reviews")})
    if rating_tag is None:
        return pd.NA

    rating = rating_tag.text.split("%")[0]
    return rating

def get_platform(page):
    classes = page.find_all("div","GameSummary_profile_info__HZFQu GameSummary_medium___r_ia")
    for classe in classes:
        if "Platform" in classe.find("strong").text:
            platform = classe.text
            platform = platform.split(":")[1][1:]
            platform = platform.split(', ')
            return platform
    return pd.NA # pas de plateforme renseignee

def get_genre(page):
    classes = page.find_all("div","GameSummary_profile_info__HZFQu GameSummary_medium___r_ia")
    for classe in classes:
        if "Genre" in classe.find("strong").text:
            genre = classe.text
            genre = genre.split(":")[1][1:]
            genre = genre.split(', ')
            return genre
    return pd.NA # pas de genre renseigne

def get_date(page):
    classes = page.find_all("div","GameSummary_profile_info__HZFQu")
    for classe in classes:
        description = classe.find("strong")
        if description != None:
            if "NA" in description.text:
                date = classe.text
                date = date.split(":")[1][1:]
                return int(date[-4:])
    return pd.NA # pas de date renseignee

def get_time(page):
    table = page.find("table","GameTimeTable_game_main_table__7uN3H")
    if table != None:
        colonne_main_story = table.find("tr","spreadsheet")
        time_main_story = colonne_main_story.find_all("td")[2]
        time_main_story = time_main_story.text
        time_main_story = time_main_story.split(" ")
        if len(time_main_story)>1:
            heures = float(time_main_story[0][:-1])
            minutes = float(time_main_story[1][:-1])
            time = heures + minutes*1/60
        else:
            time = time_main_story[0][:-1]

        return float(time)
    else:
        return pd.NA # temps non renseigné
    

In [1]:
def get_url_steam(page):
    a = page.find("a", "GameSummary_steam_text__Fe9Uv")
    if a is None:
        return 

    return a["href"]


def get_page(url):
    try:
        request_text = request.urlopen(url).read()
        page = bs4.BeautifulSoup(request_text, "lxml")
        return page
    except urllib.error.HTTPError:
        return


def get_price(steam_page):
    div = steam_page.find("strong", attrs={"class":"GameSummary_price"})

    if div is None:
        return pd.NA

    return div.text

In [39]:
def get_memoire_vive(page):
    url_steam = get_url_steam(page)
    if url_steam != None:
        page_steam = get_page(url_steam)
        div = page_steam.find("div","game_area_sys_req_full")
        if div != None:
            ul = div.find("ul","bb_ul")
            lis = ul.find_all("li")
            for li in lis:
                if li.find("strong") != None:
                    if "Memory:" == li.find("strong").text:
                        memoire_vive = li.text.split(":")[1]
                        memoire_vive_nb = float(memoire_vive.split(" ")[1])
                        unite = memoire_vive.split(" ")[2]
                        if unite == "MB":
                            memoire_vive_nb*=0.001
                        return memoire_vive_nb
        else:
            div = page_steam.find("div","game_area_sys_req_leftCol")
            if div != None:
                ul = div.find("ul","bb_ul")
                lis = ul.find_all("li")
                for li in lis:
                    if li.find("strong") != None:
                        if "Memory:" == li.find("strong").text:
                            memoire_vive = li.text.split(":")[1]
                            memoire_vive_nb = float(memoire_vive.split(" ")[1])
                            unite = memoire_vive.split(" ")[2]
                            if unite == "MB":
                                memoire_vive_nb*=0.001
                            return memoire_vive_nb
    
    return pd.NA



def get_espace_disque(page):
    url_steam = get_url_steam(page)
    if url_steam != None:
        page_steam = get_page(url_steam)
        div = page_steam.find("div","game_area_sys_req_full")
        if div != None:
            ul = div.find("ul","bb_ul")
            lis = ul.find_all("li")
            for li in lis:
                if li.find("strong") != None:
                    if "Storage:" == li.find("strong").text:
                        espace_disque = li.text.split(":")[1]
                        espace_disque_nb = float(espace_disque.split(" ")[1])
                        unite = espace_disque.split(" ")[2]
                        if unite == "MB":
                            espace_disque_nb*=0.001
                        return espace_disque_nb
        else:
            div = page_steam.find("div","game_area_sys_req_leftCol")
            if div != None:
                ul = div.find("ul","bb_ul")
                lis = ul.find_all("li")
                for li in lis: 
                    if li.find("strong") != None:
                        if "Storage:" == li.find("strong").text:
                            espace_disque = li.text.split(":")[1]
                            espace_disque_nb = float(espace_disque.split(" ")[1])
                            unite = espace_disque.split(" ")[2]
                            if unite == "MB":
                                espace_disque_nb*=0.001
                            return espace_disque_nb
        
    return pd.NA     


def get_pourcentage_pos(page):
    url_steam = get_url_steam(page)
    if url_steam != None:
        page_steam = get_page(url_steam)
        span = page_steam.find("span","nonresponsive_hidden responsive_reviewdesc")
        if span != None:
            texte = span.text
            texte = texte.split("-")[1]
            texte = texte.split("%")[0]
            pourcentage = texte[1:]
            return pourcentage
    return pd.NA


def get_steam_rating_stats(page):
    url_steam = get_url_steam(page)
    if url_steam != None:
        page_steam = get_page(url_steam)
        reviewCount = page_steam.find("meta",itemprop="reviewCount")
        if reviewCount != None:
            reviewCount = int(reviewCount.get("content"))
            ratingValue = page_steam.find("meta",itemprop="ratingValue")
            ratingValue = int(ratingValue.get("content"))
            bestRating = page_steam.find("meta",itemprop="bestRating")
            bestRating = int(bestRating.get("content"))
            worstRating = page_steam.find("meta",itemprop="worstRating")
            worstRating = int(worstRating.get("content"))
            return reviewCount,ratingValue,bestRating,worstRating
    
    return pd.NA,pd.NA,pd.NA,pd.NA

def get_steam_description(page):
    url_steam = get_url_steam(page)
    if url_steam != None:
        page_steam = get_page(url_steam)
        div = page_steam.find("div",id="game_area_description")
        descr = div.text
        if descr[20]=="\t":
            return descr[25:]
        else:
            return descr[16:]    
    return pd.NA

def get_language(page):
    url_steam = get_url_steam(page)
    if url_steam != None:
        page_steam = get_page(url_steam)
        tables = page_steam.find_all("table")
        for table in tables:
            if table["class"]==["game_language_options"]:
                trs = table.find_all("tr")
                langues_audio = []
                langues_sous_titres = []
                for tr in trs[1:]:
                    tds = tr.find_all("td")
                    if tds[2].find("span") != None:
                        langues_audio.append(tds[0].text[6:-3])
                    if tds[3].find("span") != None:
                        langues_sous_titres.append(tds[0].text[6:-3])
                if langues_audio == []:
                    langues_audio = pd.NA
                if langues_sous_titres == []:
                    langues_sous_titres = pd.NA
                return langues_audio,langues_sous_titres
    return pd.NA,pd.NA


In [40]:
df = pd.DataFrame(columns=["title","rating","platform","avg_time","genre","date","storage","memory","pourcentage","steam_description", 
                           "steam_rating","steam_review_count","steam_bestRating","steam_worstRating","languages_audio","languages_subtitles"])

# 108288
for i in range(40000, 40003):
    try:
        request_text = request.urlopen(base_url + str(i)).read()
        page = bs4.BeautifulSoup(request_text, "lxml")
    except urllib.error.HTTPError:
        continue
    
    df2 = pd.DataFrame({"title":get_title(page),"rating":get_rating(page),
                    "platform":[get_platform(page)],"avg_time":get_time(page),
                    "genre":[get_genre(page)],"date":get_date(page),"storage":get_espace_disque(page),
                    "memory":get_memoire_vive(page),"pourcentage":get_pourcentage_pos(page),
                    "steam_description":get_steam_description(page),"steam_rating":get_steam_rating_stats(page)[1],
                    "steam_review_count":get_steam_rating_stats(page)[0],"steam_bestRating":get_steam_rating_stats(page)[2],
                    "steam_worstRating":get_steam_rating_stats(page)[3],"languages_audio":[get_language(page)[0]],
                    "languages_subtitles":[get_language(page)[1]]})
    
    df = pd.concat([df,df2],ignore_index=True)
    
    
    """

    url_steam = get_url_steam(page)
    #print(url_steam)
    
    if url_steam:
        steam_page = get_page(url_steam)
        print(url_steam)
        #print(steam_page.title.text)
        if steam_page is None:
            continue

        print(get_language(page))
    """

df

C:\Users\rapha\AppData\Local\Temp\ipykernel_9512\3718068789.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df2],ignore_index=True)


,title,rating,platform,avg_time,genre,date,storage,memory,pourcentage,steam_description,steam_rating,steam_review_count,steam_bestRating,steam_worstRating,languages_audio,languages_subtitles
0,Space Rogue,<NA>,[PC],9.783333,"[Simulation, Strategy]",2016,1.000,2.0,61,Take on the role of a spaceship captain. Lead...,6,387,10,1,<NA>,"[English, Russian, German]"
1,Mario Kart Arcade GP 2,58,[Arcade],1.366667,[Racing/Driving],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Timore 5,<NA>,[PC],NaN,[Survival Horror],2016,0.387,2.0,86,Dreams Are What Our Brains Show Us In Our Slee...,9,58,10,1,NaN,NaN
